In [2]:
from collections.abc import Generator
from dataclasses import dataclass

import polars as pl
from rich import print

import renkon.api as rk
from renkon.core.model import TraitSpec, TraitSketch, RenkonType
from renkon.core.type import PrimitiveType

In [3]:
df = pl.read_csv("fish.csv").cast({"Weight": pl.Int64}).with_columns((pl.col("Weight") > 100).alias("IsLarge"))
df

Species,Weight,Length1,Length2,Length3,Height,Width,IsLarge
str,i64,f64,f64,f64,f64,f64,bool
"""Bream""",242,23.2,25.4,30.0,11.52,4.02,true
"""Bream""",290,24.0,26.3,31.2,12.48,4.3056,true
"""Bream""",340,23.9,26.5,31.1,12.3778,4.6961,true
"""Bream""",363,26.3,29.0,33.5,12.73,4.4555,true
"""Bream""",430,26.5,29.0,34.0,12.444,5.134,true
…,…,…,…,…,…,…,…
"""Smelt""",12,11.5,12.2,13.4,2.0904,1.3936,false
"""Smelt""",13,11.7,12.4,13.5,2.43,1.269,false
"""Smelt""",12,12.1,13.0,13.8,2.277,1.2558,false


In [4]:
schema = rk.Schema.from_polars(df.schema)
print(schema)

{
    'Species': StringType(),
    'Weight': IntType(),
    'Length1': FloatType(),
    'Length2': FloatType(),
    'Length3': FloatType(),
    'Height': FloatType(),
    'Width': FloatType(),
    'IsLarge': BoolType()
}

In [5]:
spec = rk.trait.Linear2.spec
print(spec)

TraitSpec(
    id='renkon.core.trait.linear',
    name='renkon.core.trait.linear',
    kind=<TraitKind.MODEL: 'model'>,
    pattern='{Y} = {b_1}*{X_1} + {b_0}',
    commutors=set(),
    typevars={},
    typings={
        'X_1': UnionType(ts=frozenset({FloatType(), IntType()})),
        'Y': UnionType(ts=frozenset({FloatType(), IntType()})),
        'b_0': FloatType(),
        'b_1': FloatType()
    }
)

In [6]:
rk.trait.Linear2.__name__

'Linear2'

In [7]:
def primitive_types() -> Generator[PrimitiveType]:
    yield rk.int_()
    yield rk.float_()
    yield rk.str_()
    yield rk.bool_()

In [8]:
print(rk.trait.Equal.spec)

TraitSpec(
    id='Equal',
    name='Equal',
    kind=<TraitKind.LOGICAL: 'logical'>,
    pattern='{A} = {B}',
    commutors={'B', 'A'},
    typevars={'T': UnionType(ts=frozenset({IntType(), StringType(), BoolType()}))},
    typings={'A': 'T', 'B': 'T'}
)

In [9]:
def monomorphize(spec: rk.TraitSpec) -> list[TraitSpec]:
    """
    :returns: a list of typing dictionaries where all type variables are replaced by concrete types.
    """
    typevars = spec.typevars

    # Split the typings out into mono (concrete) and poly (variable) parts.
    mono_typings = {k: ty for k, ty in spec.typings.items() if isinstance(ty, RenkonType)}
    poly_typings = {k: tv for k, tv in spec.typings.items() if isinstance(tv, str)}

    typings = list()
    for typevar in spec.typevars:
        print(typevar)
        
    return typings

In [10]:
monomorphize(rk.trait.Equal.spec)

T

[]

In [11]:
def instantiate(spec: rk.TraitSpec, schema: rk.Schema) -> Generator[TraitSketch]:
    @dataclass
    class Problem:
        trait_spec: TraitSpec
        df_schema: rk.Schema
        mv_tv: dict[str, str]
        tv_schema: rk.Schema

    @dataclass
    class Candidate:
        tv_ty: dict[str, str]  # tv ty choices 
        tv_schema_rest: rk.Schema  # remaining tvs 

    type Solution = TraitSketch

    def make_problem() -> Problem:
        """
        Normalizes problem so that all metavars have a type variable.
        
        :returns: a mapping from columns to type variables, and one from type variables to upper type bounds.
        """
        mv_tv = dict()
        tyvar_schema = rk.Schema(spec.typevars)

        for mv, ty in spec.typings.items():
            match ty:
                case str():
                    mv_tv[mv] = ty  # ty is a tv already
                case RenkonType():
                    tv = f"_T_{mv}"  # create a new unique tv 
                    tyvar_schema[tv] = ty
                    mv_tv[mv] = tv

        return Problem(
            trait_spec=spec,
            df_schema=schema,
            mv_tv=mv_tv,
            tv_schema=tyvar_schema
        )

    def make_root_candidate(p: Problem) -> Candidate:
        return Candidate(
            tv_ty=dict(),
            tv_schema_rest=p.tv_schema
        )

    def reject(p: Problem, c: Candidate) -> bool:
        """:returns: true if the partial bindings are not worth continuing with."""
        return False

    def accept(p: Problem, c: Candidate) -> bool:
        """:returns: true if the bindings fully satisfy the instantiation constraints."""
        return True

    def first(p: Problem, c: Candidate) -> Candidate:
        """Generate the first extension of candidate c."""
        pass

    def next(p: Problem, s: Candidate) -> Candidate:
        """Generate the next alternative extension of candidate after s."""
        pass

    def output(bindings: dict[str, str]) -> Solution:
        return TraitSketch(spec=spec, schema=schema, bindings=bindings)

    def backtrack(p: Problem, c: Candidate) -> Generator[[TraitSketch]]:
        pass

    p = make_problem()
    c = make_root_candidate(p)
    print(c)
    yield from backtrack(p, c)

In [12]:
inst = instantiate(rk.trait.Linear2.spec, schema)

print(list(inst))

Candidate(
    tv_ty={},
    tv_schema_rest={
        '_T_X_1': UnionType(ts=frozenset({FloatType(), IntType()})),
        '_T_Y': UnionType(ts=frozenset({FloatType(), IntType()})),
        '_T_b_0': FloatType(),
        '_T_b_1': FloatType()
    }
)

TypeError: 'NoneType' object is not iterable

### Notes

#### The problem:

There is a difference between `x :: int | str` and `y :: T, z :: T, T s.t. T :: int | str`:

  - `x` is allowed to be either `int` or `float`.
  - `y` and `z` are allowed to _both_ either be _the same choice_ of `int` or `float`. 

In other words, `y :: T, z:: T s.t. T :: int | str` does **not** reduce to `y :: int | str, z :: int | str`.

However, introducing type variables, `x :: T_x s.t. T_x :: int | str`, is correct.

#### Is it enough to pick concrete types first?

...or can it be any subtype? Consider `y :: T, z :: T s.t. T : int | str | bool` (ignoring `float` for now due to subtyping).

Is it okay to choose `T :: int | str`? Why or why not?

In [ ]:
def pseudo_


def pseudo_instantiate(spec: rk.TraitSpec, schema: rk.Schema) -> Generator[TraitSketch]:
    